https://adventofcode.com/2019/day/22

In [54]:
import math
from functools import partial
import random
from collections import Counter, defaultdict

import pandas as pd
import numpy as np

In [2]:
with open('data/22.txt') as fh:
    data = fh.read()

In [3]:
print(data[:200])

deal with increment 15
cut 2234
deal with increment 30
cut -1865
deal with increment 26
cut -5396
deal with increment 69
deal into new stack
deal with increment 64
cut -5111
deal with increment 23
dea


In [4]:
def cut(d, n):
    return np.concatenate((d[n:], d[:n]))

def deal(d, n):
    dl = len(d)
    e = np.empty_like(d, dtype=int)
    for i, c in enumerate(d):
        e[(i * n) % dl] = c
    return e

def newstack(d):
    return d[: : -1]

In [5]:
def parsecmd(cmd):
    cmd = cmd.strip()
    if cmd.startswith('cut'):
        n = int(cmd.split(' ')[-1])
        return partial(cut, n=n)
    elif cmd.startswith('deal with'):
        n = int(cmd.split(' ')[-1])
        return partial(deal, n=n)
    elif cmd.startswith('deal into'):
        return newstack
    else:
        raise ValueError("Can't happen: " + str(cmd))

In [6]:
def applycmds(cmds, deck):
    deck = deck.copy()
    for cmd in cmds.split('\n'):
        cmd = cmd.strip()
        if not cmd:
            continue
        deck = parsecmd(cmd)(deck)
    return deck

In [7]:
deck = np.arange(10, dtype=int)
cmds = """\
deal into new stack
cut -2
deal with increment 7
cut 8
cut -4
deal with increment 7
cut 3
deal with increment 9
deal with increment 3
cut -1
"""
applycmds(cmds, deck)

array([9, 2, 5, 8, 1, 4, 7, 0, 3, 6])

In [8]:
%%time
deck = applycmds(data, np.arange(10007, dtype=int))
deck

CPU times: user 90.7 ms, sys: 694 µs, total: 91.4 ms
Wall time: 90.2 ms


array([9497, 5065,  633, ..., 2779, 8354, 3922])

In [9]:
(deck == 2019).argmax()

7096

## Part 2

In [21]:
try:
    deck = np.arange(119315717514047, dtype=int)
except Exception as e:
    print(e)

Unable to allocate array with shape (119315717514047,) and data type int64


Tried and failed.

Wrote functions to track a position through sequential commands. This worked on deck_size = 10_007, but would have taken forever with the huge deck and gazillion iterations. Fruitlessly explored periodicity.

Finally checked https://www.reddit.com/r/adventofcode/comments/ee0rqi/2019_day_22_solutions/ and
found that the key idea is to make transformations into linear polynomials, compose into a single polynomial, then use "modpow" (pow(a,b,c) where c is modulus) to quickly apply it many many times.

See https://github.com/metalim/metalim.adventofcode.2019.python/blob/master/22_cards_shuffle.ipynb

In [76]:
deck11 = np.arange(11, dtype=int)
deck11

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [77]:
cmds = """\
deal into new stack
cut -2
deal with increment 7
"""
applycmds(cmds, deck11)

array([ 1,  4,  7, 10,  2,  5,  8,  0,  3,  6,  9])

In [80]:
def linear(x, a, b):
    return a * x + b

In [78]:
applycmds("deal into new stack", deck11)

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [81]:
# deal into new stack 
L = 11
a, b = 1, 0
a = -a
b = L - b - 1
a, b

(-1, 10)

In [82]:
linear(3, a, b)

7

In [83]:
applycmds("cut -2", deck11)

array([ 9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8])

In [99]:
# cut
L = 11
a, b = 1, 0
n = -2
b = (b + n) % L
a, b

(1, 9)

In [100]:
linear(3, a, b)

12

In [101]:
linear(3, a, b) % L

1

In [98]:
applycmds("cut 2", deck11)

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1])

In [96]:
# cut
L = 11
a, b = 1, 0
n = 2
b = (b + n) % L
a, b

(1, 2)

In [97]:
linear(3, a, b) % L

5